# NOTEBOOK 3: NONLINEAR MODEL PREDICTION
## Use trained nonlinear model to predict efficiency from F1-F15 values

In [2]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import pickle
import json
import warnings
warnings.filterwarnings('ignore')

print("🎯 NONLINEAR MODEL PREDICTION FOR CEMENT PLANT EFFICIENCY")
print("=" * 60)

🎯 NONLINEAR MODEL PREDICTION FOR CEMENT PLANT EFFICIENCY


In [3]:
# Cell 2: Load Trained Model and Metadata
try:
    # Load the trained model
    with open('best_nonlinear_model.pkl', 'rb') as f:
        model = pickle.load(f)
    print("✅ Loaded trained nonlinear model")
    
    # Load model metadata
    with open('nonlinear_model_data.json', 'r') as f:
        model_data = json.load(f)
    print("✅ Loaded model metadata")
    
    # Load scaler if needed
    scaler = None
    uses_scaling = model_data['uses_scaled_features']
    if uses_scaling:
        with open('feature_scaler.pkl', 'rb') as f:
            scaler = pickle.load(f)
        print("✅ Loaded feature scaler")
    
except FileNotFoundError as e:
    print(f"❌ Required file not found: {e}")
    print("Please run 02_nonlinear_model_training.ipynb first!")
    raise

# Display model information
print(f"\n📊 MODEL INFORMATION:")
print(f" Model: {model_data['model_name']}")
print(f" Test R²: {model_data['test_r2']:.4f}")
print(f" Expected MAE: ±{model_data['test_mae']:.2f}% efficiency points")
print(f" Uses feature scaling: {'Yes' if uses_scaling else 'No'}")
print(f" Training samples: {model_data['training_samples']}")
print(f" Target range: {model_data['target_range'][0]:.1f}% to {model_data['target_range'][1]:.1f}%")

✅ Loaded trained nonlinear model
✅ Loaded model metadata
✅ Loaded feature scaler

📊 MODEL INFORMATION:
 Model: SVR (RBF Kernel)
 Test R²: 0.9997
 Expected MAE: ±0.06% efficiency points
 Uses feature scaling: Yes
 Training samples: 4320
 Target range: 55.0% to 77.0%


In [4]:
# Cell 3: Define Feature Bounds and Prediction Function
print(f"\n📊 FEATURE BOUNDS ANALYSIS:")
print(f"{'Feature':<8} {'Min':<12} {'Max':<12} {'Range':<12} {'Mean':<12}")
print("-" * 60)

# Define explicit bounds dictionary matching feature_names order
explicit_bounds = {
    'F1': (800, 1200),       # Feed Size
    'F2': (20, 25),          # Product Size
    'F3': (1600, 2000),      # Mill Power Consumption (first)
    'F4': (80, 120),         # Mill Inlet Temperature
    'F5': (85, 95),          # Blending Efficiency
    'F6': (850, 900),        # C5 Temperature
    'F7': (85, 90),          # Heat Recovery Efficiency
    'F8': (4, 6),            # Fuel Flow Rate
    'F9': (8, 12),           # Primary Fuel Flow
    'F10': (800, 1100),      # Secondary Air Temperature
    'F11': (800, 1200),      # Kiln Drive Power
    'F12': (1200, 1400),     # Clinker Inlet Temperature
    'F13': (400, 600),       # Cooling Air Flow
    'F14': (2000, 2500),     # Mill Power Consumption (second)
    'F15': (8, 15),          # Packing Rate
}

bounds = []
feature_info = {}
feature_names = model_data['feature_names']

for i, feature in enumerate(feature_names):
    # Use explicit bounds
    min_val, max_val = explicit_bounds[feature]
    range_val = max_val - min_val
    mean_val = (min_val + max_val) / 2  # Use midpoint as approximate mean
    
    bounds.append((min_val, max_val))
    feature_info[feature] = {
        'min': min_val,
        'max': max_val,
        'range': range_val,
        'mean': mean_val
    }
    
    print(f"{feature:<8} {min_val:<12.2f} {max_val:<12.2f} {range_val:<12.2f} {mean_val:<12.2f}")

print(f"\nBounds created for all {len(bounds)} features")

# Define prediction function
def predict_efficiency(x):
    """Predict efficiency from feature vector x"""
    x = np.array(x).reshape(1, -1)
    
    if uses_scaling:
        x_scaled = scaler.transform(x)
        return model.predict(x_scaled)[0]
    else:
        return model.predict(x)[0]

# Test prediction function
test_x = [feature_info[feature]['mean'] for feature in feature_names]
test_pred = predict_efficiency(test_x)
print(f"\n🧪 Prediction function test:")
print(f" Input: Mean feature values")
print(f" Predicted efficiency: {test_pred:.2f}%")
print(f" ✅ Prediction function working correctly")


📊 FEATURE BOUNDS ANALYSIS:
Feature  Min          Max          Range        Mean        
------------------------------------------------------------
F1       800.00       1200.00      400.00       1000.00     
F2       20.00        25.00        5.00         22.50       
F3       1600.00      2000.00      400.00       1800.00     
F4       80.00        120.00       40.00        100.00      
F5       85.00        95.00        10.00        90.00       
F6       850.00       900.00       50.00        875.00      
F7       85.00        90.00        5.00         87.50       
F8       4.00         6.00         2.00         5.00        
F9       8.00         12.00        4.00         10.00       
F10      800.00       1100.00      300.00       950.00      
F11      800.00       1200.00      400.00       1000.00     
F12      1200.00      1400.00      200.00       1300.00     
F13      400.00       600.00       200.00       500.00      
F14      2000.00      2500.00      500.00       2250.00  

In [5]:
# Cell 4: HARDCODED INPUT VALUES - MODIFY THESE AS NEEDED
print("\n🎛️ INPUT PARAMETER VALUES:")
print("=" * 50)

# ✅ MODIFY THESE VALUES TO TEST DIFFERENT SCENARIOS
# Current values are set to approximate midpoints within bounds
input_values = {
    'F1': 1200.0,    # Feed Size (800-1200)
    'F2': 22.5,      # Product Size (20-25)
    'F3': 1800.0,    # Mill Power Consumption 1 (1600-2000)
    'F4': 100.0,     # Mill Inlet Temperature (80-120)
    'F5': 90.0,      # Blending Efficiency (85-95)
    'F6': 875.0,     # C5 Temperature (850-900)
    'F7': 87.5,      # Heat Recovery Efficiency (85-90)
    'F8': 5.0,       # Fuel Flow Rate (4-6)
    'F9': 10.0,      # Primary Fuel Flow (8-12)
    'F10': 950.0,    # Secondary Air Temperature (800-1100)
    'F11': 1000.0,   # Kiln Drive Power (800-1200)
    'F12': 1300.0,   # Clinker Inlet Temperature (1200-1400)
    'F13': 500.0,    # Cooling Air Flow (400-600)
    'F14': 2250.0,   # Mill Power Consumption 2 (2000-2500)
    'F15': 11.5      # Packing Rate (8-15)
}

# Display input values
print(f"{'Parameter':<8} {'Value':<10} {'Bounds':<20} {'Status'}")
print("-" * 50)

for feature in feature_names:
    value = input_values[feature]
    min_bound, max_bound = explicit_bounds[feature]
    bounds_str = f"[{min_bound:.1f}, {max_bound:.1f}]"
    
    # Check if value is within bounds
    if min_bound <= value <= max_bound:
        status = "✅ Valid"
    else:
        status = "❌ Out of bounds"
    
    print(f"{feature:<8} {value:<10.1f} {bounds_str:<20} {status}")

print(f"\n📝 To change values, modify the 'input_values' dictionary above and rerun this cell")


🎛️ INPUT PARAMETER VALUES:
Parameter Value      Bounds               Status
--------------------------------------------------
F1       1200.0     [800.0, 1200.0]      ✅ Valid
F2       22.5       [20.0, 25.0]         ✅ Valid
F3       1800.0     [1600.0, 2000.0]     ✅ Valid
F4       100.0      [80.0, 120.0]        ✅ Valid
F5       90.0       [85.0, 95.0]         ✅ Valid
F6       875.0      [850.0, 900.0]       ✅ Valid
F7       87.5       [85.0, 90.0]         ✅ Valid
F8       5.0        [4.0, 6.0]           ✅ Valid
F9       10.0       [8.0, 12.0]          ✅ Valid
F10      950.0      [800.0, 1100.0]      ✅ Valid
F11      1000.0     [800.0, 1200.0]      ✅ Valid
F12      1300.0     [1200.0, 1400.0]     ✅ Valid
F13      500.0      [400.0, 600.0]       ✅ Valid
F14      2250.0     [2000.0, 2500.0]     ✅ Valid
F15      11.5       [8.0, 15.0]          ✅ Valid

📝 To change values, modify the 'input_values' dictionary above and rerun this cell


In [6]:
# Cell 5: Make Prediction
print("\n🔮 EFFICIENCY PREDICTION:")
print("=" * 40)

# Prepare input vector
input_vector = [input_values[feature] for feature in feature_names]

# Validate all inputs are within bounds
all_valid = True
violations = []

for i, (feature, value) in enumerate(input_values.items()):
    min_bound, max_bound = explicit_bounds[feature]
    if not (min_bound <= value <= max_bound):
        all_valid = False
        violations.append(f"{feature}: {value} not in [{min_bound}, {max_bound}]")

if not all_valid:
    print("❌ INPUT VALIDATION FAILED:")
    for violation in violations:
        print(f"   {violation}")
    print("\n⚠️ Please correct input values and rerun")
else:
    print("✅ All input values are within valid bounds")
    
    # Make prediction
    try:
        predicted_efficiency = predict_efficiency(input_vector)
        
        print(f"\n🎯 PREDICTION RESULT:")
        print(f" Predicted Efficiency: {predicted_efficiency:.4f}%")
        print(f" Rounded: {predicted_efficiency:.2f}%")
        
        # Compare with target and ranges
        target_77 = 77.0
        model_min = model_data['target_range'][0]
        model_max = model_data['target_range'][1]
        model_mean = model_data['target_mean']
        
        print(f"\n📊 CONTEXT:")
        print(f" Target goal: ≥{target_77:.1f}%")
        print(f" Historical range: {model_min:.1f}% to {model_max:.1f}%")
        print(f" Historical mean: {model_mean:.1f}%")
        
        print(f"\n🏆 ASSESSMENT:")
        if predicted_efficiency >= target_77:
            print(f" ✅ TARGET ACHIEVED! Efficiency ≥ {target_77}%")
            margin = predicted_efficiency - target_77
            print(f" 📈 Margin above target: +{margin:.2f}% points")
        else:
            print(f" ❌ Target not reached. Efficiency < {target_77}%")
            gap = target_77 - predicted_efficiency
            print(f" 📉 Gap to target: -{gap:.2f}% points")
        
        # Performance relative to historical data
        if predicted_efficiency > model_mean:
            diff = predicted_efficiency - model_mean
            print(f" 📈 Above historical average by +{diff:.2f}% points")
        else:
            diff = model_mean - predicted_efficiency
            print(f" 📉 Below historical average by -{diff:.2f}% points")
        
        # Model confidence
        expected_mae = model_data['test_mae']
        print(f"\n🔬 MODEL CONFIDENCE:")
        print(f" Expected error: ±{expected_mae:.2f}% (MAE)")
        print(f" Confidence interval: [{predicted_efficiency-expected_mae:.2f}%, {predicted_efficiency+expected_mae:.2f}%]")
        
    except Exception as e:
        print(f"❌ PREDICTION ERROR: {e}")
        print("Please check that the model files are correctly loaded")


🔮 EFFICIENCY PREDICTION:
✅ All input values are within valid bounds

🎯 PREDICTION RESULT:
 Predicted Efficiency: 69.9576%
 Rounded: 69.96%

📊 CONTEXT:
 Target goal: ≥77.0%
 Historical range: 55.0% to 77.0%
 Historical mean: 61.7%

🏆 ASSESSMENT:
 ❌ Target not reached. Efficiency < 77.0%
 📉 Gap to target: -7.04% points
 📈 Above historical average by +8.26% points

🔬 MODEL CONFIDENCE:
 Expected error: ±0.06% (MAE)
 Confidence interval: [69.90%, 70.02%]


In [7]:
# Cell 6: Sensitivity Analysis (Optional)
print("\n🔍 SENSITIVITY ANALYSIS:")
print("=" * 35)
print("Analyzing how small changes in each parameter affect efficiency...")

if all_valid:
    base_prediction = predict_efficiency(input_vector)
    
    print(f"\n{'Parameter':<8} {'Current':<10} {'Change +5%':<12} {'Effect':<12} {'Change -5%':<12} {'Effect'}")
    print("-" * 70)
    
    sensitivity_data = []
    
    for i, feature in enumerate(feature_names):
        current_value = input_values[feature]
        
        # Test +5% change
        test_vector_plus = input_vector.copy()
        test_vector_plus[i] = current_value * 1.05
        
        # Test -5% change
        test_vector_minus = input_vector.copy()
        test_vector_minus[i] = current_value * 0.95
        
        try:
            pred_plus = predict_efficiency(test_vector_plus)
            pred_minus = predict_efficiency(test_vector_minus)
            
            effect_plus = pred_plus - base_prediction
            effect_minus = pred_minus - base_prediction
            
            sensitivity_data.append({
                'feature': feature,
                'sensitivity': abs(effect_plus) + abs(effect_minus)
            })
            
            print(f"{feature:<8} {current_value:<10.2f} {pred_plus:<12.3f} {effect_plus:+.3f} {pred_minus:<12.3f} {effect_minus:+.3f}")
            
        except:
            print(f"{feature:<8} {current_value:<10.2f} {'Error':<12} {'---':<12} {'Error':<12} {'---'}")


🔍 SENSITIVITY ANALYSIS:
Analyzing how small changes in each parameter affect efficiency...

Parameter Current    Change +5%   Effect       Change -5%   Effect
----------------------------------------------------------------------
F1       1200.00    66.944       -3.014 72.976       +3.019
F2       22.50      67.898       -2.060 70.138       +0.180
F3       1800.00    69.586       -0.371 68.850       -1.107
F4       100.00     69.208       -0.749 69.662       -0.296
F5       90.00      65.594       -4.364 66.416       -3.541
F6       875.00     63.511       -6.447 64.901       -5.056
F7       87.50      65.641       -4.317 66.239       -3.719
F8       5.00       70.156       +0.199 69.416       -0.541
F9       10.00      70.199       +0.241 68.891       -1.067
F10      950.00     68.799       -1.158 70.067       +0.110
F11      1000.00    69.325       -0.633 70.195       +0.238
F12      1300.00    68.173       -1.784 68.213       -1.745
F13      500.00     69.468       -0.490 69.945   

In [8]:
# Cell 7: Quick Scenario Testing
print("\n🎲 QUICK SCENARIO TESTING:")
print("=" * 35)

# Define a few test scenarios
scenarios = {
    'Conservative': {  # Lower values within bounds
        'F1': 900, 'F2': 21, 'F3': 1700, 'F4': 90, 'F5': 87,
        'F6': 860, 'F7': 86, 'F8': 4.5, 'F9': 9, 'F10': 900,
        'F11': 900, 'F12': 1250, 'F13': 450, 'F14': 2100, 'F15': 10
    },
    'Aggressive': {   # Higher values within bounds
        'F1': 1150, 'F2': 24, 'F3': 1950, 'F4': 115, 'F5': 93,
        'F6': 890, 'F7': 89, 'F8': 5.8, 'F9': 11.5, 'F10': 1050,
        'F11': 1150, 'F12': 1380, 'F13': 580, 'F14': 2450, 'F15': 14
    },
    'Balanced': {     # Midpoint values
        'F1': 1000, 'F2': 22.5, 'F3': 1800, 'F4': 100, 'F5': 90,
        'F6': 875, 'F7': 87.5, 'F8': 5, 'F9': 10, 'F10': 950,
        'F11': 1000, 'F12': 1300, 'F13': 500, 'F14': 2250, 'F15': 11.5
    }
}

print(f"{'Scenario':<12} {'Predicted Efficiency':<20} {'Target Met'}")
print("-" * 45)

for scenario_name, scenario_values in scenarios.items():
    scenario_vector = [scenario_values[feature] for feature in feature_names]
    
    try:
        scenario_pred = predict_efficiency(scenario_vector)
        target_met = "✅ YES" if scenario_pred >= 77.0 else "❌ NO"
        print(f"{scenario_name:<12} {scenario_pred:<20.3f} {target_met}")
    except Exception as e:
        print(f"{scenario_name:<12} {'Error':<20} {'---'}")

print(f"\n💡 Use these scenarios as starting points for optimization")


🎲 QUICK SCENARIO TESTING:
Scenario     Predicted Efficiency Target Met
---------------------------------------------
Conservative 62.598               ❌ NO
Aggressive   60.858               ❌ NO
Balanced     76.453               ❌ NO

💡 Use these scenarios as starting points for optimization


In [9]:
# Cell 8: Save Prediction Results
if all_valid:
    print("\n💾 SAVING PREDICTION RESULTS:")
    
    # Create results dictionary
    prediction_results = {
        'model_name': model_data['model_name'],
        'prediction_timestamp': pd.Timestamp.now().isoformat(),
        'input_parameters': input_values,
        'predicted_efficiency': float(predicted_efficiency),
        'target_77_achieved': bool(predicted_efficiency >= 77.0),
        'expected_error_mae': model_data['test_mae'],
        'model_performance': {
            'test_r2': model_data['test_r2'],
            'cv_r2': model_data['cv_r2']
        },
        'parameter_bounds': explicit_bounds,
        'all_parameters_valid': all_valid
    }
    
    # Save to JSON
    with open('prediction_results.json', 'w') as f:
        json.dump(prediction_results, f, indent=2)
    
    print("✅ prediction_results.json saved")
    print("\n📋 Results Summary:")
    print(f" - Input parameters: {len(input_values)} features")
    print(f" - Predicted efficiency: {predicted_efficiency:.3f}%")
    print(f" - Target achieved: {'Yes' if predicted_efficiency >= 77.0 else 'No'}")
    print(f" - Expected accuracy: ±{model_data['test_mae']:.2f}%")
else:
    print("\n⚠️ Cannot save results due to invalid input parameters")


💾 SAVING PREDICTION RESULTS:
✅ prediction_results.json saved

📋 Results Summary:
 - Input parameters: 15 features
 - Predicted efficiency: 69.958%
 - Target achieved: No
 - Expected accuracy: ±0.06%


In [10]:
# Cell 9: Summary and Next Steps
print("\n" + "=" * 60)
print("🎉 NONLINEAR MODEL PREDICTION COMPLETE!")
print("=" * 60)

print(f"\n📊 PREDICTION SUMMARY:")
if all_valid:
    print(f" Model Used: {model_data['model_name']}")
    print(f" Input Parameters: 15 features (F1-F15)")
    print(f" Predicted Efficiency: {predicted_efficiency:.3f}%")
    print(f" Target (77%): {'✅ Achieved' if predicted_efficiency >= 77.0 else '❌ Not achieved'}")
    print(f" Model Confidence: R² = {model_data['test_r2']:.4f}")
    print(f" Expected Error: ±{model_data['test_mae']:.2f}% (MAE)")
else:
    print(" ❌ Prediction failed due to invalid input parameters")

print(f"\n🔄 HOW TO USE THIS NOTEBOOK:")
print(" 1. Modify values in Cell 4 'input_values' dictionary")
print(" 2. Ensure all values are within specified bounds")
print(" 3. Rerun Cell 4 and Cell 5 to get new predictions")
print(" 4. Use Cell 6 for sensitivity analysis")
print(" 5. Compare different scenarios in Cell 7")

print(f"\n📁 OUTPUT FILES:")
if all_valid:
    print(" ✅ prediction_results.json - Complete prediction results")
else:
    print(" ⚠️ No output files created (invalid inputs)")

print(f"\n🎯 PARAMETER BOUNDS REMINDER:")
for feature, (min_val, max_val) in explicit_bounds.items():
    print(f" {feature}: [{min_val}, {max_val}]")

print(f"\n✅ Ready to predict cement plant efficiency for any F1-F15 combination!")


🎉 NONLINEAR MODEL PREDICTION COMPLETE!

📊 PREDICTION SUMMARY:
 Model Used: SVR (RBF Kernel)
 Input Parameters: 15 features (F1-F15)
 Predicted Efficiency: 69.958%
 Target (77%): ❌ Not achieved
 Model Confidence: R² = 0.9997
 Expected Error: ±0.06% (MAE)

🔄 HOW TO USE THIS NOTEBOOK:
 1. Modify values in Cell 4 'input_values' dictionary
 2. Ensure all values are within specified bounds
 3. Rerun Cell 4 and Cell 5 to get new predictions
 4. Use Cell 6 for sensitivity analysis
 5. Compare different scenarios in Cell 7

📁 OUTPUT FILES:
 ✅ prediction_results.json - Complete prediction results

🎯 PARAMETER BOUNDS REMINDER:
 F1: [800, 1200]
 F2: [20, 25]
 F3: [1600, 2000]
 F4: [80, 120]
 F5: [85, 95]
 F6: [850, 900]
 F7: [85, 90]
 F8: [4, 6]
 F9: [8, 12]
 F10: [800, 1100]
 F11: [800, 1200]
 F12: [1200, 1400]
 F13: [400, 600]
 F14: [2000, 2500]
 F15: [8, 15]

✅ Ready to predict cement plant efficiency for any F1-F15 combination!
